
The project duration will be from September 2022 to December 2022. I will use Python to extract the data from Google news, including headings, passage and publication date, and download the historical bitcoin price from January 2017 to August 2022 on Yahoo Finance. For the language of the news, English news will only be used for the convenience to analyse the text. For the data cleaning part, incorrectly formatted, duplicate, and incomplete data would be removed within a dataset, especially combining numerous data sources. On top of that, some news websites may well block the program from continuously scrapping the text data. This missing data should also be handled by dropping the observations or inputting the missing values according to other observations. The sentiment score will be obtained using Python natural language processing (NLP) libraries and own analysis. For example, if the text has the words like “bullish”, “optimistic” and “gain”, it will have a positive sentiment score. If the text has the words like “bearish” and “loss”, it will have a negative sentiment score.

Install all required libraries including selenium, vaderSentiment, matplotlib and plotly

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.



Import all required libraries

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

#from selenium import webdriver
#import webbrowser

from selenium.webdriver import Chrome
from selenium.webdriver import ActionChains
from selenium import webdriver
import time


from selenium.webdriver.common.by import By
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer



import pandas as pd
import plotly.express as px
import numpy as np



import inspect

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import csv



Read the csv file downloaded from Yahoo Finance (https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD)

In [3]:


bitcoin_price_monthly = pd.read_csv ('bitcoin_price_monthly.csv')
bitcoin_price_weekly = pd.read_csv ('bitcoin_price_weekly.csv')
bitcoin_price_daily = pd.read_csv ('bitcoin_price_daily.csv')





The sentiment_scores() calculates the sentiment score of the text




The returnmonth() returns the year, month, day from a link

The updatedate() retzurns the next day

The changelink() update the link

The returnlink() returns a link with a given year, month and day.

In [5]:
def sentiment_scores(sentence):
 
    sid_obj = SentimentIntensityAnalyzer()
 
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict['compound']


def returnmonth(link):
    min_month = 0
    min_day = 0
    min_year = 0
    max_month = 0
    max_day = 0
    max_year = 0
    
    min_month = int(link[link.find("min%3A")+6:link.find("%2F")])
    min_day = int(link[link.find("%2F")+3:link.find("%2F20", 173)])
    min_year = int(link[link.find("%2F20", 173)+3:link.find("%2Ccd_max")])
    #max_month = int(link[link.find("max%3A")+6:link.find("%2F", 179)])
    #max_day = int(link[link.find("%2F", 179)+3:link.find("%2F20", 185)])
    #max_year = int(link[link.find("%2F20", 185)+3:link.find("&tbm=nws")])

    
    return min_month, min_day, min_year



def updatedate(min_month, min_day, min_year, max_month, max_day, max_year):
    if min_month == 12 and min_day == 31:
        min_monh = 1
        max_month = 1
        
        min_day = 1
        max_day = 1 
        
        min_year = min_year + 1
        max_year = max_year + 1
        
    elif (min_month == 1 or min_month == 3 or min_month == 5 or min_month == 7 or min_month == 8 or min_month == 10 or min_month == 12) and min_day == 31:
        min_month = min_month + 1
        max_month = max_month + 1
        
        min_day = 1
        max_day = 1 
        
    elif (min_month == 4 or min_month == 6 or min_month == 9 or min_month == 11) and min_day == 30:
        min_month = min_month + 1
        max_month = max_month + 1
        
        min_day = 1
        max_day = 1 
        
    elif min_month == 2 and min_day == 29 and (min_year == 2020 or min_year == 2016 or min_year == 2012 or min_year == 2008):
        min_month = min_month + 1
        max_month = max_month + 1
        
        min_day = 1
        max_day = 1 
        
    elif min_month == 2 and min_day == 28:
        min_month = min_month + 1
        max_month = max_month + 1
        
        min_day = 1
        max_day = 1 
    
    else:
        min_day = min_day + 1
        max_day = max_day + 1
        
    return str(min_month), str(min_day), str(min_year), str(max_month), str(max_day), str(max_year)


def changelink(link):


    month = int(link[link.find("min%3A")+6:link.find("%2F")])
    day = int(link[link.find("%2F")+3:link.find("%2F20", 173)])
    year = int(link[link.find("%2F20", 173)+3:link.find("%2Ccd_max")])
    #max_month = int(link[link.find("max%3A")+6:link.find("%2F", 179)])
    #max_day = int(link[link.find("%2F", 179)+3:link.find("%2F20", 185)])
    #max_year = int(link[link.find("%2F20", 185)+3:link.find("&tbm=nws")])
    
    
    result = "https://www.google.com/search?q=bitcoin&lr=lang_en&hl=en-US&biw=1440&bih=706&sxsrf=ALiCzsaLHN0RAHh1o4D2QSeTTNQeMdakdw%3A1660888361013&source=lnt&tbs=cdr%3A1%2Ccd_min%3A" + updatedate(month, day, year, month,day, year)[0] + "%2F" + updatedate(month, day, year, month, day, year)[1]+ "%2F"+ updatedate(month, day, year, month, day, year)[2] +"%2Ccd_max%3A"+ updatedate(month, day, year, month, day, year)[3] + "%2F"+ updatedate(month, day, year, month, day, year)[4] + "%2F"+ updatedate(month, day, year, month, day, year)[5] + "&tbm=nws"
    
    return result

def returnlink(month, day, year):
    result = "https://www.google.com/search?q=bitcoin&lr=lang_en&hl=en-US&biw=1440&bih=706&sxsrf=ALiCzsZ-yYrB04X2_aorDONhp1sBBArpwQ%3A1660887991435&source=lnt&tbs=cdr%3A1%2Ccd_min%3A"+str(month)+"%2F"+str(day)+"%2F"+str(year)+"%2Ccd_max%3A"+str(month)+"%2F"+str(day)+"%2F"+str(year)+"&tbm=nws"
    
    return result



It will build the list

In [11]:
headline_list = []
day_list = []
day_list_2 = []
year_list = []
month_list = []
month_list_2 = []
sentiment_list = []
link_list = []


year_month_list = []
year_month_day_list = []




It will open an automated Google Chrome browser

In [12]:

driver = Chrome('./chromedriver')


/var/folders/mr/lclw3hlj0zs374kh0pvrp2zm0000gn/T/ipykernel_3108/801403462.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('./chromedriver')



It will web scrap the news title, publication date and news link from Google. It will start from 2017/1/1 to 2022/08/31. However, the programme may not be able to bypass the captcha test.


In [21]:


#start from 2017/1/1



nex_page_link = ""

year = 2017
month = 1
day = 1

page = 10


link = returnlink(month, day, year)


for j in range(0, page):
    
    month = returnmonth(link)[0]
    day = returnmonth(link)[1]
    year = returnmonth(link)[2]

    driver.get(link)
    bs = BeautifulSoup(driver.page_source, 'html.parser')

    search_result_list = bs.find_all('div', {"class": "vJOb1e aIfcHf Hw13jc"})
    search_result_list_2 = bs.find_all('td', {"class": "d6cvqb BBwThe"})
    search_result_list_3 = bs.find_all('a', {'class':'WlydOe'})
    
    time.sleep(1)


    for i in range(0, len(search_result_list)):
        headline_list.append(search_result_list[i].find('div', {"class": "mCBkyc y355M ynAwRc MBeuO nDgy9d"}).get_text())
        link_list.append(search_result_list_3[i].get('href'))
        month_list.append(month)
        day_list.append(day)
        year_list.append(year)
        
    
    ##working 
    while str(search_result_list_2).find("pnnext") != -1:
        next_page_link = str("https://www.google.com" + search_result_list_2[-1].find("a", {"id": "pnnext"}).get('href'))
        driver.get(next_page_link)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        search_result_list = bs.find_all('div', {"class": "vJOb1e aIfcHf Hw13jc"})
        search_result_list_2 = bs.find_all('td', {"class": "d6cvqb BBwThe"})
        search_result_list_3 = bs.find_all('a', {'class':'WlydOe'})
        
        for k in range(0, len(search_result_list)):
            headline_list.append(search_result_list[k].find('div', {"class": "mCBkyc y355M ynAwRc MBeuO nDgy9d"}).get_text())
            link_list.append(search_result_list_3[k].get('href'))
            month_list.append(month)
            day_list.append(day)
            year_list.append(year)
        
        #time.sleep(3)
        
    link = changelink(link)
    
print('done')
    



    


/var/folders/mr/lclw3hlj0zs374kh0pvrp2zm0000gn/T/ipykernel_3108/3548259835.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('./chromedriver')


done



It will clean the data and add the sentiment score.

In [22]:
day_list_2 = []
month_list_2 = []

year_month_list = []
year_month_day_list = []


for i in range(0, len(day_list)):
    if int(day_list[i]) < 10:
        day_list_2.append("0" + str(day_list[i]))
    else:
        day_list_2.append(str(day_list[i]))
        
for i in range(0, len(month_list)):
    if int(month_list[i]) < 10:
        month_list_2.append("0" + str(month_list[i]))
    else:
        month_list_2.append(str(month_list[i]))
        

        
for i in range(0, len(month_list_2)):
    year_month_list.append(str(year_list[i])+str(month_list_2[i]))
    year_month_day_list.append(str(year_list[i])+str(month_list_2[i])+str(day_list_2[i]))
        


sentiment_list = []


for i in range(0, len(headline_list)):
    sentiment_list.append(sentiment_scores(headline_list[i]))


It will build the dataframe by combining the year, month, day, headline, link, sentiment score etc.

In [24]:


text_analysis_df = pd.DataFrame({'year': year_list, 'month': month_list, 'day': day_list, 'year_month':year_month_list, 'year_month_day': year_month_day_list, 'healine': headline_list, 'link': link_list, 'sentiment': sentiment_list})  




It remove all duplicates news

In [25]:

text_analysis_df = text_analysis_df.drop_duplicates(subset ="healine", keep = 'first')


It prints the dataframe

In [26]:

text_analysis_df

,year,month,day,year_month,year_month_day,healine,link,sentiment
0,2022,8,25,202208,20220825,"Bitcoin (BTC), Ethereum (ETH) Volatility Leads...",https://www.bloomberg.com/news/articles/2022-0...,0.0000
1,2022,8,25,202208,20220825,"Bitcoin, Ethereum Technical Analysis: ETH Back...",https://news.bitcoin.com/bitcoin-ethereum-tech...,0.0000
2,2022,8,25,202208,20220825,Bitcoin Bearish Signal: Miners Continue To Dump,https://www.newsbtc.com/news/bitcoin/bitcoin-b...,-0.3818
3,2022,8,25,202208,20220825,Analysis: The $885 million Bitcoin Depot SPAC ...,https://www.ledgerinsights.com/analysis-the-88...,0.0000
4,2022,8,25,202208,20220825,Bitcoin Depot to list in the U.S. through $885...,https://www.reuters.com/markets/deals/bitcoin-...,0.0000
...,...,...,...,...,...,...,...,...
1344,2022,8,31,202208,20220831,"Bitcoin Price Chart May Signal End to Selloff,...",https://www.inferse.com/197765/bitcoin-price-c...,0.0000
1345,2022,8,31,202208,20220831,Crypto will become an inflation hedge — just n...,https://techbullion.com/crypto-will-become-an-...,0.0000
1346,2022,8,31,202208,20220831,Singapore Considers Stricter Rules for Retail ...,https://www.inferse.com/197692/singapore-consi...,0.0000
1347,2022,8,31,202208,20220831,Pimride (PIM) Is Now Available for Trading on ...,https://www.livebitcoinnews.com/pimride-pim-is...,0.0000



It export the dataframe to csv

In [28]:

text_analysis_df.to_csv('bitcoin_news.csv', index = False)
